# Evaluate the models that are saved in the models folder

# Imports

removed checking if in CoLab

In [ ]:
!pip install -U scikit-image

In [ ]:
#imports
from platform import python_version

#basic python stuff
import os
import json
from pathlib import Path

#basics from the SciPy Stack
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#colab stuff
from google.colab import drive

#data managing
from sklearn.model_selection import train_test_split
from skimage import io #read in images
from skimage.transform import resize

#model
%tensorflow_version 2.x
import tensorflow as tf

# evaluation
import sklearn

#progress bar
from tqdm.notebook import tqdm

In [ ]:
# settings

# implements progress_apply into pandas
tqdm.pandas(desc='Pandas_Progress')

In [ ]:
print("Tensorflow version", tf.__version__)
print("Python version =",python_version())

In [ ]:
# get access to drive
drive.mount('/content/drive')

In [ ]:
# import local python files
import importlib.util

# https://github.com/maxvfischer/keras-image-segmentation-loss-functions
spec = importlib.util.spec_from_file_location("binary_losses", "/content/drive/MyDrive/ML_Project_Satellite_Images/binary_losses.py")
binary_losses = importlib.util.module_from_spec(spec)
spec.loader.exec_module(binary_losses)

# Data Constants

In [ ]:
# data preprocessing
img_size = 128
anti_aliasing = True
mask_threshold = 0.5

# for loading the model
model_name = 'large_skip_5000_128px_32_filters_04_dropout'

custom_objects_dict = None # use this normally
#custom_objects_dict = {'loss':binary_losses.binary_balanced_cross_entropy(beta=1.)} # use this for custom loss functions

# Retrieve the Dataset

In [ ]:
# unzip data
!unzip -n -q /content/drive/MyDrive/ML_Project_Satellite_Images/data/current_dataset.zip -d /content/

In [ ]:
# read in samples.csv with information about the images
samples_df = pd.read_csv('/content/dataset/samples.csv')
samples_df.set_index('id', inplace=True)
samples_df

In [ ]:
# paths to the sat/mask folder
path_sat_folder = '/content/dataset/images/satellite/'
path_mask_folder = '/content/dataset/images/mask/'

In [ ]:
# append absoulute paths of the images to the dataframe
samples_df['abs_satellite_path'] = samples_df['satellite_file'].apply(lambda x: path_sat_folder+x)
samples_df['abs_mask_path'] = samples_df['mask_file'].apply(lambda x: path_mask_folder+x)

# Functions for Image reading and plotting

In [ ]:
# function for image reading
def read_satellite_img(filepath):
  img = io.imread(filepath)
  img = resize(img, output_shape=(img_size,img_size), anti_aliasing=anti_aliasing, preserve_range=True)
  img = img / 255.
  return img

def read_mask_img(filepath):
  img = io.imread(filepath)
  if len(img.shape) > 2:
    img = img[:,:,2]
  img = resize(img, output_shape=(img_size,img_size), anti_aliasing=anti_aliasing, preserve_range=True)
  img = img / 255.
  mask = img > mask_threshold
  img[mask] = 1
  img[~mask] = 0
  return img

In [ ]:
# function to load a batch of images
def load_img_batch(samples_df,ids):
  satellite_imgs = samples_df.loc[ids,'abs_satellite_path'].progress_apply(read_satellite_img)
  mask_imgs = samples_df.loc[ids,'abs_mask_path'].progress_apply(read_mask_img)

  satellite_imgs = np.stack(satellite_imgs.to_numpy())
  mask_imgs = np.stack(mask_imgs.to_numpy())

  return satellite_imgs, mask_imgs

In [ ]:
# function to show some samples (with or without the predictions)
def show_sample(X, Y, samples_df, ids, Y_pred=None, threshold=None, sample_size=10, fig_height=6):
  rnd_sample_indices = np.random.random_integers(low=0,high=X.shape[0]-1,size=sample_size)
  cols = 2 if Y_pred is None else 3
  if Y_pred is None:
    cols = 2
  elif threshold is not None:
    cols = 4
  else:
    cols = 3

  for i in rnd_sample_indices:
    fig, axs = plt.subplots(1,cols, figsize=(fig_height*cols,fig_height))
    axs[0].set_title(f'Country: {samples_df.loc[ids[i],"country"]}')
    axs[0].imshow(X[i])
    axs[1].set_title('Given Mask')
    axs[1].imshow(Y[i])
    if Y_pred is not None:
      axs[2].set_title('Prediction')
      axs[2].imshow(Y_pred[i])
      if threshold is not None:
        Y_pred_mask = Y_pred[i] >= threshold
        Y_pred[i,Y_pred_mask] = 1
        Y_pred[i,~Y_pred_mask] = 0
        axs[3].set_title(f'Prediction with threshold = {threshold}')
        axs[3].imshow(Y_pred[i])

    for ax in axs:
      ax.set_xticks([])
      ax.set_yticks([])
    fig.tight_layout()

In [ ]:
def show_images(X,Ys,names,fig_height=4):
  'Plots X and multiple Y'
  if not isinstance(Ys,list):
    Ys = [Ys]
  if not isinstance(names,list):
    names = [names]
  cols = 1 + len(Ys)
  for i in range(X.shape[0]):
    fig,axs = plt.subplots(1,cols,figsize=(fig_height*cols,fig_height))
    axs[0].axis('off')
    axs[0].imshow(X[i])
    axs[0].set_title('Satellite')
    for j,(Y,name) in enumerate(zip(Ys,names)):
      axs[j+1].axis('off')
      axs[j+1].imshow(Y[i])
      axs[j+1].set_title(name)
    plt.show()

# Predict Masks with an alternative algorithm

In [ ]:
sample_size = 10
test_ids = np.random.choice(samples_df.index.to_numpy(), size=sample_size)
X_test, Y_test = load_img_batch(samples_df, test_ids)

## HOG

In [ ]:
from skimage.feature import hog
from skimage import exposure

In [ ]:
def calculate_hog(X):
  Y_hog = np.zeros_like(X)[...,0]
  fds = []
  for i in range(X.shape[0]):
    fd, hog_image = hog(X[i], orientations=8, pixels_per_cell=(8, 8),
                    cells_per_block=(1, 1), visualize=True, feature_vector=False, multichannel=True)
    hog_image_rescaled = exposure.rescale_intensity(hog_image, in_range=(0, 10), out_range=(0,1))
    Y_hog[i] = hog_image_rescaled
    fds.append(fd)
  fds = np.stack(fds)
  return fds, Y_hog

In [ ]:
fds, Y_hog = calculate_hog(X_test)
fds.shape, Y_hog.shape

In [ ]:
Y_hog_sum = np.sum(fds[...,0,0,:],axis=3)
Y_hog_sum.shape

In [ ]:
Ys = [Y_hog, Y_hog_sum]
names = ['HOG', 'HOG Sum']
show_images(X_test,Ys,names,fig_height=3)

## manual gradient

In [ ]:
def calculate_gradient(X):
  Y_gradient = np.stack(np.gradient(X,axis=(1,2)))
  Y_gradient_norm = np.linalg.norm(Y_gradient,axis=0)
  return Y_gradient_norm

In [ ]:
Y_grad = calculate_gradient(X_test)
Y_grad.shape

In [ ]:
from skimage.color import rgb2gray

In [ ]:
Y_grad_gray = rgb2gray(Y_grad)
Y_grad_gray.shape

In [ ]:
grad_threshold = 0.01
Y_grad_mask = Y_grad_gray < grad_threshold

In [ ]:

Ys = [Y_grad, Y_grad_gray, Y_grad_mask, Y_test]
names = ['Gradient', 'Gradient Gray', 'Gradient Mask', 'Mask']
show_images(X_test,Ys,names)

# Colors based

In [ ]:
sample_size = 100
sample_ids = np.random.choice(samples_df.index.to_numpy(), size=sample_size)

In [ ]:
train_ids, test_ids = train_test_split(sample_ids, test_size=0.5)

In [ ]:
X_train, Y_train = load_img_batch(samples_df, train_ids)
X_test, Y_test = load_img_batch(samples_df, test_ids)

# Evaluate on Test Data

In [ ]:
sample_size = 1000
test_ids_sample = np.random.choice(test_ids, size=sample_size)
X_test, Y_test = load_img_batch(samples_df, test_ids_sample)

In [ ]:
# get predictions from the model
Y_pred = model.predict(X_test, verbose=1).astype(np.float64).reshape(-1, img_size, img_size)

In [ ]:
Y_pred.min(), Y_pred.max()

In [ ]:
score = model.evaluate(X_test, Y_test, verbose=1)

In [ ]:
print(f'Test Loss: {score[0]}')
print(f'Test Accuracy: {score[1]}')

In [ ]:
Y_test.dtype, Y_pred.dtype, Y_test.shape, Y_pred.shape

### choose threshold

In [ ]:
len_thresh = 10
threshold_range = np.linspace(0.2, 0.9, len_thresh)

results_iou = np.zeros(len_thresh)
results_acc = np.zeros(len_thresh)

for i, thresh in enumerate(threshold_range):
  Y_mask_temp = Y_pred >= thresh
  Y_temp = np.zeros_like(Y_pred)
  Y_temp[Y_mask_temp] = 1

  #iou score
  intersection = np.logical_and(Y_test, Y_temp)
  union = np.logical_or(Y_test, Y_temp)
  results_iou[i] = np.sum(intersection) / np.sum(union)

  #accuracy
  results_acc[i] = np.sum(Y_temp == Y_test)/Y_temp.size

print(f'max. iou for threshold = {threshold_range[np.argmax(results_iou)]}: \t', np.max(results_iou))
print(f'max. acc for threshold = {threshold_range[np.argmax(results_acc)]}: \t', np.max(results_acc))

In [ ]:
fig, (ax1, ax2) = plt.subplots(2,1)
ax1.plot(threshold_range, results_iou)
ax1.set_title('iou_score')
ax1.set_xlabel('threshold')
ax1.set_ylabel('iou_score')
ax1.grid()

ax2.plot(threshold_range, results_acc)
ax2.set_title('accuracy')
ax2.set_xlabel('threshold')
ax2.set_ylabel('accuracy')
ax2.grid()

fig.tight_layout()
fig.show()

In [ ]:
show_sample(X_test, Y_test, samples_df, test_ids_sample, Y_pred, threshold=0.5, sample_size=50, fig_height=20)